In [1]:
import ast
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
from utils import SRC_DIR
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet50_Weights
from torch.utils.data import Dataset, DataLoader
from nn_utils import WeightedBinaryCrossEntropyLoss
from sklearn.metrics import (
    precision_score, recall_score,
    roc_auc_score, average_precision_score,
    hamming_loss, accuracy_score,
    coverage_error, f1_score
)

In [2]:
csv_path = './datasets/cropped_all_one_hot.csv'
data = pd.read_csv(csv_path, index_col=0)

# convert string labels to list
data['labels'] = data['labels'].apply(ast.literal_eval)
data = data[data.labels.apply(lambda x: len(x) > 5)]

# Extract image paths and labels
data['file_names'] = data['file_path'].apply(lambda x: x.split("/")[-1]).tolist()
data['galleries'] = data['file_path'].apply(lambda x: x.split("/")[-3]).tolist()
data.drop(columns=['file_path', 'labels'], inplace=True)
data.shape

(256465, 91)

In [3]:
labels = []
file_paths = []
gallery_names = data['galleries'].unique()
grouped_data = data.groupby('galleries')

max_seq_len = 0
for gallery in tqdm(gallery_names, total=len(gallery_names), desc='Processing galleries'):
    temp_df = grouped_data.get_group(gallery)
    file_paths.append(temp_df['file_names'].tolist())
    max_seq_len = max(max_seq_len, len(temp_df))
    temp_labels = temp_df.drop(columns=['file_names', 'galleries']).sum(axis=0).values
    labels.append((temp_labels > 0).astype(int))

Processing galleries: 100%|██████████| 16253/16253 [00:02<00:00, 5539.69it/s]


In [4]:
labels = np.array(labels)
class_names = data.drop(['file_names', 'galleries'], axis=1).columns.tolist()  # Save the class names
num_classes = len(class_names)

# Save class names to a file for future reference
with open('class_names.txt', 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

print(f"Class names: {class_names}")

Class names: ['anal sex', 'anilingus', 'asian', 'ass', 'bath', 'bdsm', 'beach', 'big breasts', 'big woman', 'bikini', 'black penis', 'blonde', 'blowjob', 'bondage', 'boots', 'brunette', 'butt plug', 'chubby', 'clothed', 'cosplay', 'cowgirl (sex position)', 'creampie', 'cum in mouth', 'cumshot', 'cunnilingus', 'curly', 'curvy', 'dildo', 'doggy style (sex position)', 'double penetration', 'dress', 'ebony', 'facial', 'feet', 'fellatio', 'fisting', 'footjob', 'glasses', 'granny', 'group sex', 'gym', 'hairy vulva', 'handjob', 'heels', 'intercourse', 'interracial', 'jeans', 'kissing', 'latina', 'leather', 'lesbian', 'lingerie', 'maid', 'masturbation', 'mature', 'milf', 'missionary (sex position)', 'naked breasts', 'nurse', 'outdoor', 'panties', 'pantyhose', 'petite', 'pissing', 'pool', 'public', 'redhead', 'sandals', 'selfie', 'sex toys', 'shorts', 'shower', 'skinny', 'skirt', 'small breasts', 'smoking', 'socks', 'sports', 'stockings', 'tattoo', 'teacher', 'teen', 'thick', 'thong', 'threesom

In [5]:
# Calculate the number of occurrences for each class
class_counts = np.sum(labels, axis=0)
total_samples = labels.shape[0]

# Calculate the weights
class_weights = total_samples / (num_classes * class_counts)

# Convert to a PyTorch tensor
pos_weight = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
from sklearn.model_selection import train_test_split

file_paths_train, file_paths_val, labels_train, labels_val = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42
)

In [ ]:
class AlbumDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None, max_seq_len=16):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        album_paths = self.file_paths[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        # Apply transformations
        processed_album = []
        for img_path in album_paths:
            img = Image.open(SRC_DIR / img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            processed_album.append(img)

        # Padding if necessary
        if len(processed_album) < self.max_seq_len:
            padding = [torch.zeros_like(processed_album[0])] * (self.max_seq_len - len(processed_album))
            processed_album.extend(padding)
        else:
            processed_album = processed_album[:self.max_seq_len]  # Trim if too long

        processed_album = torch.stack(processed_album)
        length = min(len(album_paths), self.max_seq_len)

        return processed_album, label, length

In [ ]:
def collate_fn(batch):
    images, labels, lengths = zip(*batch)

    # Pad sequences to the maximum length in this batch
    max_len = max(lengths)
    padded_images = []
    for img_seq in images:
        if len(img_seq) < max_len:
            padding = [torch.zeros_like(img_seq[0])] * (max_len - len(img_seq))
            img_seq = torch.cat([img_seq, torch.stack(padding)])
        padded_images.append(img_seq)

    padded_images = torch.stack(padded_images)
    labels = torch.stack(labels)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return padded_images, labels, lengths

In [ ]:
# Data augmentation and preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = AlbumDataset(file_paths_train, labels_train, transform=transform)
val_dataset = AlbumDataset(file_paths_val, labels_val, transform=transform)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [ ]:
class MultilabelAlbumModel(nn.Module):
    def __init__(self, num_classes, max_seq_len, lstm_hidden_size=512, lstm_layers=2):
        super(MultilabelAlbumModel, self).__init__()
        self.max_seq_len = max_seq_len
        self.cnn = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.cnn.fc = nn.Identity()
        self.rnn = nn.LSTM(input_size=2048, hidden_size=lstm_hidden_size, num_layers=lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(lstm_hidden_size * 2, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x, lengths):
        batch_size, seq_len, c, h, w = x.size()
        cnn_features = []
        for t in range(seq_len):
            img = x[:, t, :, :, :]
            feature = self.cnn(img)
            cnn_features.append(feature)
        cnn_features = torch.stack(cnn_features, dim=1)
        packed_input = nn.utils.rnn.pack_padded_sequence(cnn_features, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (h_n, c_n) = self.rnn(packed_input)
        rnn_out, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True, total_length=self.max_seq_len)
        idx = (lengths - 1).view(-1, 1).expand(len(lengths), rnn_out.size(2)).unsqueeze(1)
        last_output = rnn_out.gather(1, idx).squeeze(1)
        out = self.fc1(last_output)
        out = self.fc2(out)
        return torch.sigmoid(out)

In [ ]:
def calculate_metrics(outputs, targets):
    outputs = outputs.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    outputs = (outputs > 0.5).astype(int)

    precision = precision_score(targets, outputs, average='micro')
    recall = recall_score(targets, outputs, average='micro')
    macro_f1 = f1_score(targets, outputs, average='macro')
    micro_f1 = f1_score(targets, outputs, average='micro')
    roc_auc = roc_auc_score(targets, outputs, average='micro')
    pr_auc = average_precision_score(targets, outputs, average='micro')
    hamming = hamming_loss(targets, outputs)
    subset_acc = accuracy_score(targets, outputs)
    coverage = coverage_error(targets, outputs)

    return {
        "precision": precision,
        "recall": recall,
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "hamming_loss": hamming,
        "subset_accuracy": subset_acc,
        "coverage_error": coverage
    }

In [ ]:
# Example training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MultilabelAlbumModel(num_classes=num_classes, max_seq_len=max_seq_len)
model.to(device)

# Custom loss function
criterion = WeightedBinaryCrossEntropyLoss(pos_weight=pos_weight)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_outputs = []
    all_labels = []

    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for batch in train_dataloader:
            images, labels, lengths = batch
            images = images.to(device)
            labels = labels.to(device)
            lengths = lengths.to(device)

            optimizer.zero_grad()
            outputs = model(images, lengths)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            all_outputs.append(outputs)
            all_labels.append(labels)

            # Update progress bar
            pbar.update(1)

    epoch_loss = running_loss / len(train_dataloader)
    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)

    train_metrics = calculate_metrics(all_outputs, all_labels)

    model.eval()
    val_loss = 0.0
    val_outputs = []
    val_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            images, labels, lengths = batch
            images = images.to(device)
            labels = labels.to(device)
            lengths = lengths.to(device)

            outputs = model(images, lengths)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            val_outputs.append(outputs)
            val_labels.append(labels)

    val_loss /= len(val_dataloader)
    val_outputs = torch.cat(val_outputs)
    val_labels = torch.cat(val_labels)

    val_metrics = calculate_metrics(val_outputs, val_labels)

    print(
        f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Train Micro F1: {train_metrics["micro_f1"]:.4f}, Val Micro F1: {val_metrics["micro_f1"]:.4f}')
    print(
        f"Train Precision: {train_metrics['precision']:.4f}, Train Recall: {train_metrics['recall']:.4f}, Train Macro F1: {train_metrics['macro_f1']:.4f}")
    print(
        f"Val Precision: {val_metrics['precision']:.4f}, Val Recall: {val_metrics['recall']:.4f}, Val Macro F1: {val_metrics['macro_f1']:.4f}")
    print(
        f"Val ROC AUC: {val_metrics['roc_auc']:.4f}, Val PR AUC: {val_metrics['pr_auc']:.4f}, Val Hamming Loss: {val_metrics['hamming_loss']:.4f}")
    print(
        f"Val Subset Accuracy: {val_metrics['subset_accuracy']:.4f}, Val Coverage Error: {val_metrics['coverage_error']:.4f}")